In [6]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Drugs table

In [26]:
drugs = pd.read_excel('bcdrugs.xlsx')
drugs.shape

(1203, 36)

Drugs columns = 'Drug ID', 'Drug Name', 'CAS Number', 'Chemical Name',
       'New Molecular Entity', 'Drug Interactions', 'Comparative Efficacy',
       'Highest Development Phase', 'Indication', 'Molecular Formula',
       'Mechanism Of Action', 'Organisation',
       'Pharmacodynamic Characteristics', 'Route Of Administration',
       'Route Of Elimination', 'Pharmacokinetic Characteristics',
       'Adverse Events', 'Therapeutic Class (EphMRA)',
       'Therapeutic Class (WHO)', 'Synonyms/Brand Names',
       'Antimicrobial Activity', 'Immunogenicity',
       'Last Significant Update Date', 'Therapeutic Area',
       'Orphan Designation', 'License Availability', 'Brands', 'On Fastrack',
       'Fastrack country', 'On PRIME', 'PRIME country', 'Biomarkers-Inclusion',
       'Biomarkers-Exclusion', 'Biomarkers-Outcome Measure',
       'Biomarkers-Safety', 'Biomarkers-not specified'

In [81]:
drugs['Highest Development Phase'].value_counts()

Marketed                                     175
Preclinical                                  167
Phase II                                     149
No development reported (Preclinical)         87
Phase III                                     84
Discontinued (II)                             84
Phase I                                       65
Discontinued (Preclinical)                    60
Discontinued (III)                            46
Phase I/II                                    44
Research                                      35
Discontinued (I)                              25
No development reported (Research)            21
No development reported (I)                   21
No development reported (II)                  20
Registered                                    15
Phase II/III                                  14
Discontinued (I/II)                           11
Discontinued (Preregistration)                10
No development reported (Clinical)            10
Preregistration     

## Trials table

In [7]:
trials = pd.read_excel('bcdrugsct.xlsx')
trials.shape

(4529, 41)

In [159]:
cols = ['Trial ID','Mechanism of action', 'Drug class (therapeutic effect)', 'Drug class (chemical)', \
        'Indication', 'Organisations', 'Trial Design', 'Location', 'Phase of Trial', 'Subject Age',\
        'Planned Subject Number', 'Trial Centre Details', 'Lead Centre', 'Trial Initiation date', \
        'Trial End date', 'Trial Status', 'Trial History', 'Diseases treated', 'Primary Drugs']
smaller_trials = trials[cols]
date_cols = ['Phase of Trial','Trial Initiation date', 'Trial End date', 'trial_start','trial_end']

In [133]:
df['Trial Initiation date'].head()

1      01 Dec 2013 (actual)
10     01 Dec 2015 (actual)
16     12 Feb 2019 (actual)
478    06 Dec 2018 (actual)
481    01 Feb 2014 (actual)
Name: Trial Initiation date, dtype: object

In [ ]:

def phase_1(drug_name):
    '''
    take earliest Phase I date and latest Phase II
    '''
    tucatinib_trials = trials[trials['Primary Drugs']==drug_name]
    trial_phases = ['Phase I', 'Phase II', 'Phase III']
    cleaned_tucatinib_trials = tucatinib_trials[tucatinib_trials['Phase of Trial'].isin(trial_phases)]
    phase_dummies = pd.get_dummies(cleaned_tucatinib_trials['Phase of Trial'])
    cleaned_tucatinib_trials_dummies = cleaned_tucatinib_trials.join(phase_dummies)
    #cleaned_tucatinib_trials_dummies[['Trial Initiation date', 'Trial End date']]
    df = cleaned_tucatinib_trials.join(phase_dummies)

    #not DRY as the answers to make multiargument apply functions involved more complicated things than I wanted.
    def start_to_datetime(x):
        date_str = re.findall('[\d\s\w]*', x['Trial Initiation date'])[0].strip()
        if date_str == '':
            return False
        else:
            return pd.to_datetime(date_str)

    def end_to_datetime(x):
        date_str = re.findall('[\d\s\w]*', x['Trial End date'])[0].strip()
        if date_str == '':
            return False
        else:
            return pd.to_datetime(date_str)

    df['trial_start'] = df.apply(start_to_datetime, axis=1)
    df['trial_end'] = df.apply(end_to_datetime, axis=1)

    t_phase_1_min = min(list(df[(df['trial_start']!=False) & (df['Phase of Trial']=='Phase I')]['trial_start'].values))
    t_phase_1_max = max(list(df[(df['trial_start']!=False) & (df['Phase of Trial']=='Phase I')]['trial_start'].values))

    return f'min is {t_phase_1_min} and max is {t_phase_1_max}'

phase_1('Xentuzumab')


# df['split_start_date'] = df['Trial Initiation date'].apply(lambda x: x.split(' '))
# df['split_end_date'] = df['Trial End date'].apply(lambda x: x.split(' '))
# df2 = df[df['split_start_date'].apply(lambda x: len(x[:3]))==3]
# df3 = df2[df2['split_end_date'].apply(lambda x: len(x[:3]))==3]
# df3[date_cols]
#df3.apply(lambda x: ' '.join(x['split_start_date']))
# in_progress_trials = trials[trials['split_start_date'].apply(lambda x: len(x[:3]))==3]
# planned_trials = trials[trials['split_start_date'].apply(lambda x: len(x[:3]))==2]
# # one_split = trials['split_end_date'].values[0]
# # pd.to_datetime(' '.join(one_split[:3]))







In [52]:
phase_two = smaller_trials[smaller_trials['Phase of Trial'] == 'Phase II']
phase_three = smaller_trials[smaller_trials['Phase of Trial'] == 'Phase III']

In [63]:
unique_drugs =list(smaller_trials['Primary Drugs'].unique())

In [ ]:
trials[trials['Primary Drugs']=='Tucatinib']

In [ ]:
def trial_history_to_table(th):
    rows = str.split(th, ';')
    #columns_trial_1 = [str.split(i, '|') for i in rows_trial_1]
    table = [str.split(row, '|') for row in rows]
    return pd.DataFrame(table,columns=['Date','Status','Notes'])

trial_history_to_table(phase_three['Trial History'].values[0])